In [5]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from tqdm import tqdm
import json
from sklearn.metrics import pairwise_distances
sys.path.append("../")
from utils import prewhiten, l2_normalize
from tqdm import tqdm
import json
import tensorflow as tf
import numpy as np

In [6]:
model_path = '/home/ivan/pascal_adversarial_faces/keras-facenet/model/facenet_keras.h5'
model = tf.keras.models.load_model(model_path)

In [18]:
def load_one_facenet(path, crop_box=None, resize_size=None, prewhiten_img=True):
    img = Image.open(path).convert("RGB")

    if crop_box:
        img = img.crop(crop_box)

    if resize_size:
        img = img.resize((resize_size, resize_size))

    img = (np.array(img)).astype(np.float32)

    if prewhiten_img:
        img = prewhiten(img)

    return img

def load_folder_for_facenet(folder, crop=False, resize_size=None):
    paths_list = glob.glob(folder, self.person_name + "*")

    final_imgs = []
    for img_path in paths_list:

    
        final_imgs.append(
            load_one_facenet(
                img_path, crop_box=None, prewhiten=False, resize_size=resize_size))

    final_imgs = np.array(final_imgs)
    mean, std = np.mean(final_imgs), np.std(final_imgs)
    final_imgs = prewhiten(np.array(final_imgs))
    return final_imgs, mean, std

In [32]:
class FacesDatabase:
    def __init__(self, model, image_directory, attack_strategy, epsilon):
        self.model = model
        self. associated_identities = []
        self.vectors = []
        for person_name in os.listdir(image_directory):
            self.add_images_for_person(image_directory, attack_strategy, person_name, epsilon)

            
    def add_folder(self, folder_path, person_name, from_indx, to_indx):
        '''
        add all images at folder_path between alphabetically sorted from_indx and to_indx
        to person of person_name in the Azure group
        '''
        # List all jpeg, jpg and png images
        # glob returns full paths
        file_paths = [fn for fn in glob.glob(
            os.path.join(folder_path, '*')
        ) if fn.endswith("png") or fn.endswith("jpg") or fn.endswith("jpeg")]

        # Restrict to the first limit instances only in alphabetical order
        file_paths = sorted(file_paths)[from_indx:to_indx]

        print(f"Adding folder {folder_path}")
        # Add to Azure instance
        for img_path in tqdm(file_paths):
            self.vectors.append(self.compute_vector(img_path))
            self.associated_identities.append(person_name)

            
    def compute_vector(self, img_path):
        img = load_one_facenet(img_path)
        return np.array(l2_normalize(self.model(np.expand_dims(img, axis=0)))[0])
        
        
    def add_images_for_person(
            self, image_directory, attack_strategy, person_name, epsilon):
        # Get all the protected identities.
        # Remember our folder structure is ground_truth_identity/attack_strategy/protected_identity/epsilon_X/png/*.png
        folders_wildcard = os.path.join(image_directory, person_name, attack_strategy, "*")
        protected_folders = glob.glob(folders_wildcard)

        if len(protected_folders) < 1:
            print(f"For folder {folders_wildcard} no protected folders")
            return

        # 1. Add clean images truly belonging to this identity
        # When epsilon = 0.0, we have clean images and it doesn't matter which identity is "being protected"
        # as they are all unmodified images but duplicated in each identity.
        clean_folder = os.path.join(protected_folders[0], "epsilon_0.0", "png")

        # For now, hard code that we take only 1  clean image - the first one alphabetically.
        self.add_folder(clean_folder, person_name, 0, 1)

        # 2. Add decoy images belonging to this person_name in reality but modified to protect protected identities
        # For each identity, get one decoy at a different index.
        for indx, protected_identity_folder in enumerate(protected_folders):
            full_folder_path = os.path.join(
                protected_identity_folder, f"epsilon_{epsilon}", "png")
            self.add_folder(full_folder_path, person_name, indx + 1, indx + 2)

    
    def nearest_neighbor_to_img_at_path(self, query_path):
        query_vector = self.compute_vector(query_path)
        distances = pairwise_distances([query_vector], self.vectors)[0]
        return self.associated_identities[np.argmin(distances)]
        

In [36]:
def measure_local_recall(
    face_database,
    image_directory,
):
    discovery = []
    true = []
    identified_as = []

    for protector in os.listdir(image_directory):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            f"{image_directory}/{protector}/community_naive_mean/{protected}/epsilon_0.0/png/*"
        ))

    
        top_identity = faces_database.nearest_neighbor_to_img_at_path(query_photos_paths[11])
        true.append(protector)
        identified_as.append(top_identity)


        if top_identity == protector:
            discovery.append(1.0)
        else:
            discovery.append(0.0)

    
    for true_id, recognized_id in zip(true, identified_as):
        print(f"Face of {true_id} identitifed as {recognized_id}")
        
    return sum(discovery)/len(discovery)

In [34]:
faces_database = FacesDatabase(model, "/data/vggface/test_perturbed_sampled", "robust_community_naive_mean", 0.5)


  0%|          | 0/1 [00:00<?, ?it/s]

Adding folder /data/vggface/test_perturbed_sampled/n001781/robust_community_naive_mean/n009232/epsilon_0.0/png
Adding folder /data/vggface/test_perturbed_sampled/n001781/robust_community_naive_mean/n009232/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n001781/robust_community_naive_mean/n000958/epsilon_0.5/png


  0%|          | 0/1 [00:00<?, ?it/s]

Adding folder /data/vggface/test_perturbed_sampled/n001781/robust_community_naive_mean/n003356/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n001781/robust_community_naive_mean/n008655/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n001781/robust_community_naive_mean/n008613/epsilon_0.5/png


  0%|          | 0/1 [00:00<?, ?it/s]

...

Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n005427/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n002763/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n002503/epsilon_0.5/png


  0%|          | 0/1 [00:00<?, ?it/s]

Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n003215/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n005359/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n005303/epsilon_0.5/png


100%|██████████| 1/1 [00:00<00:00, 12.07it/s]

Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n007548/epsilon_0.5/png
Adding folder /data/vggface/test_perturbed_sampled/n009114/robust_community_naive_mean/n000029/epsilon_0.5/png


In [37]:
measure_local_recall(
    faces_database,
    "/data/vggface/test_perturbed_sampled",
)

Face of n001781 identitifed as n004658
Face of n009232 identitifed as n005359
Face of n000958 identitifed as n002647
Face of n003356 identitifed as n009114
Face of n008655 identitifed as n005427
Face of n008613 identitifed as n009232
Face of n004658 identitifed as n002503
Face of n001683 identitifed as n001683
Face of n002647 identitifed as n009114
Face of n009288 identitifed as n005359
Face of n005427 identitifed as n005359
Face of n002763 identitifed as n000029
Face of n002503 identitifed as n009288
Face of n003215 identitifed as n007548
Face of n005359 identitifed as n005359
Face of n005303 identitifed as n009232
Face of n007548 identitifed as n009232
Face of n000029 identitifed as n008613
Face of n009114 identitifed as n005303


0.10526315789473684